# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
from IPython.display import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import tensorflow as tf
import tensorflow_datasets as tfds
warnings.filterwarnings("ignore")

# 14.6 텐서플로 추정기

그전에는 데코레이터를 사용해 함수를 계산 그래프로 명시적으로 컴파일하고 케라스 API를 사용해 사용자 정의 층을 추가한 피드포워드 신경망을 구현했다

텐서플로 추정기를 사용해 훈련, 예측, 평가와 같은 머신 러닝 작업의 기본 단계를 캡슐화한다.

## 14.6.1 특성 열 사용

In [3]:
Image(url='https://git.io/JL56E', width=700)

In [4]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", 
                                       ("http://archive.ics.uci.edu/ml/machine-learning-databases"
                                        "/auto-mpg/auto-mpg.data"))

column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
                'Weight', 'Acceleration', 'ModelYear', 'Origin']

df = pd.read_csv(dataset_path, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

40960/30286 [========================================] - 0s 2us/step


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
#전처리
# NA 열 삭제
df=df.dropna()
df=df.reset_index(drop=True)
df

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1


In [7]:
# 훈련/테스트 분할
import sklearn
import sklearn.model_selection

df_train,df_test=sklearn.model_selection.train_test_split(df,train_size=0.8)

train_stats=df_train.describe().transpose()

train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.403834,7.691882,9.0,17.6,22.0,29.0,46.6
Cylinders,313.0,5.488818,1.691050,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,195.316294,103.797752,68.0,105.0,151.0,267.0,455.0
Horsepower,313.0,104.587859,38.706317,46.0,76.0,92.0,125.0,230.0
Weight,313.0,2988.578275,837.863741,1649.0,2245.0,2835.0,3609.0,4997.0
Acceleration,313.0,15.492652,2.655747,8.0,14.0,15.5,17.0,23.7
ModelYear,313.0,76.102236,3.719633,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.555911,0.787058,1.0,1.0,1.0,2.0,3.0


In [8]:
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm,df_test_norm=df_train.copy(),df_test.copy()

for col_name in numeric_column_names:
  mean=train_stats.loc[col_name,'mean']
  std=train_stats.loc[col_name,'std']
  df_train_norm.loc[:,col_name]=(df_train_norm.loc[:,col_name]-mean)/std
  df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
19,26.0,-0.880410,-0.947191,-1.513651,-1.376809,1.885477,70,2
184,27.0,-0.880410,-0.908654,-0.557735,-0.938790,-0.072541,76,2
86,13.0,1.484984,1.490241,1.044071,1.192821,-0.938588,73,1
7,14.0,1.484984,2.357312,2.852561,1.579519,-2.633027,70,1
193,29.0,-0.880410,-1.062800,-1.358638,-1.138107,2.525598,76,1


In [10]:
#수치형 열
numeric_features=[]
for col_name in numeric_column_names:
  numeric_features.append(tf.feature_column.numeric_column(key=col_name))

numeric_features

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [13]:
# 순서가 있는 범주형

feature_year=tf.feature_column.numeric_column(key='ModelYear')

bucketized_features=[]

bucketized_features.append(
    tf.feature_column.bucketized_column(
        source_column=feature_year,
        boundaries=[73,76,79]
    )
)

bucketized_features

[BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79))]

In [14]:
# 순서가 없는 범주 특성

feature_origin=tf.feature_column.categorical_column_with_vocabulary_list(
    key='Origin',
    vocabulary_list=[1,2,3]
)

feature_origin

VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0)

In [16]:
categorical_indicator_features=[]
categorical_indicator_features.append(tf.feature_column.indicator_column(feature_origin))
categorical_indicator_features

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]

## 14.6.2 사전에 준비된 추정기로 머신 러닝 수행

1. 데이터 로드를 위해 입력 함수 정의

2. 데이터셋을 특성 열로 변환

3. 추정기 객체를 만듭니다. (사전 준비된 추정기를 사용하거나 케라스 모델을 추정기로 변환하여 새로 만든다.)

4. 추정기의 train,evaluate, predict 메서드를 사용

Auto MPG 예제를 사용하겠다.

첫 번째로 데이터를 처리하는 함수 정의

이 함수는 입력 특성과 레이블을 담은 튜플로 구성된 텐서플로 데이터셋을 반환한다.

특성은 딕셔너리로 구성되어야 하고 이 딕셔너리 키는 특성 열의 이름과 맞아야 한다.

In [17]:
def train_input_fn(df_train,batch_size=8):
  df=df_train.copy()
  train_x,train_y=df,df.pop('MPG')
  dataset=tf.data.Dataset.from_tensor_slices((dict(train_x),train_y))

  return dataset.shuffle(1000).repeat().batch(batch_size)

In [18]:
ds=train_input_fn(df_train_norm)
batch=next(iter(ds))
print('Keys:',batch[0].keys())

Keys: dict_keys(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'ModelYear', 'Origin'])


In [19]:
print('ModelYear:', batch[0]['ModelYear'])

ModelYear: tf.Tensor([82 81 81 78 82 80 81 73], shape=(8,), dtype=int64)


In [20]:
def eval_input_fn(df_test,batch_size=8):
  df=df_test.copy()
  test_x,test_y=df,df.pop('MPG')
  dataset=tf.data.Dataset.from_tensor_slices(
      (dict(test_x),test_y))
  return dataset.batch(batch_size)

두 번째 단계 : 특성 열 정의 (연속적인 특성을 담은 리스트, 버킷으로 나눈 특성 열을 위한 리스트, 범주 특성 열을 위한 리스트)

In [23]:
all_feature_columns=(
    numeric_features+
    bucketized_features+
    categorical_indicator_features
)

세 번째 단계 : 추정기 객체 만들기

MPG 값을 예측하는 것은 전형적인 회귀 문제

두 개의 은닉층을 사용하고 첫 번째 은닉층은 32개의 유닛, 두 번째 은닉층은 열 개의 유닛을 갖는다.

In [24]:
regressor=tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32,10],
    model_dir='models/autompg-dnnregressor/'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


추정기 장점 중 하나가 훈련하는 동안 자동으로 모델의 체크포인트를 저장하는 것

모델 훈련이 중지되었을 때도 마지막으로 저장한 체크포인트를 로드하여 훈련을 이어갈 수 있다.

세 개의 기본 단계가 끝나면 추정기를 사용하여 훈련, 평가, 예측을 수행할 수 있다.

In [25]:
EPOCHS=1000
BATCH_SIZE=8
total_steps=EPOCHS * int(np.ceil(len(df_train)/BATCH_SIZE))

print('훈련 스텝:',total_steps)

훈련 스텝: 40000


In [27]:
regressor.train(
    input_fn=lambda:train_input_fn(
        df_train_norm,batch_size=BATCH_SIZE), steps=total_steps
    )

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into models/autompg-dnnregressor/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 601.0128, step = 0
INFO:tensorflow:global_step/sec: 358.249
INFO:tensorflow:loss = 673.42175, step = 100 (0.283 sec)
INFO:tensorflow:global_step/sec: 494.338
INFO:tensorflow:loss = 809.6716, step = 200 (0.202 sec)
INFO:tensorflow:global

In [28]:
reloaded_regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    warm_start_from='models/autompg-dnnregressor/',
    model_dir='models/autompg-dnnregressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [30]:
eval_results = reloaded_regressor.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-04T04:23:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.53595s
INFO:tensorflow:Finished evaluation at 2022-01-04-04:23:43
INFO:tensorflow:Saving dict for global step 40000: average_loss = 17.968729, global_step = 40000, label/mean = 23.612656, loss = 18.213522, prediction/mean = 23.39626
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 40000: models/autompg-dnnregressor/model.ckpt-40000


In [31]:
print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

평균 손실 17.9687


In [32]:
pred_res=regressor.predict(input_fn=lambda: eval_input_fn(
    df_test_norm,batch_size=8
))

print(next(iter(pred_res)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([27.665434], dtype=float32)}


다른 추정기인 부스티드 트리 회귀 모델을 사용해 보겠다.

In [33]:
boosted_tree=tf.estimator.BoostedTreesRegressor(
    feature_columns=all_feature_columns,
    n_batches_per_layer=20,
    n_trees=200
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbch7iu1n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Plea

In [34]:
boosted_tree.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE))

eval_results = boosted_tree.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

print(eval_results)

print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpbch7iu1n/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners after saving checkp

## 14.6.3 추정기를 사용해 MNIST 손글씨 숫자 분류

In [3]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
steps_per_epoch = np.ceil(60000 / BATCH_SIZE)

In [4]:
#입력 이미지는 원래 [0,255]인 uint8 타입이기에 tf.float32로 변환해야 한다.

def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(
        image, tf.float32)
    image = tf.reshape(image, (-1,))

    return {'image-pixels':image}, label[..., tf.newaxis]

1단계 : 입력 함수 두 개를 정의 (하나는 훈련용, 다른 하나는 평가용)

In [10]:
def train_input_fn():
  datasets=tfds.load(name='mnist')
  mnist_train=datasets['train']

  dataset=mnist_train.map(preprocess)
  dataset=dataset.shuffle(BUFFER_SIZE)
  dataset=dataset.batch(BATCH_SIZE)
  return dataset.repeat()

def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

2단계 : 특성 열을 정의한다.

In [6]:
image_feature_column=tf.feature_column.numeric_column(key='image-pixels',shape=(28*28))

3단계 : 새로운 추정기를 만든다. 두 개의 은닉층 지정할 건데 첫 번째에는 32개의 유닛, 두 번째에는 16개의 유닛

In [8]:
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='models/mnist-dnn/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


4단계 : 추정기를 사용하여 훈련, 평가, 예측 수행

In [11]:
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

eval_result = dnn_classifier.evaluate(
    input_fn=eval_input_fn)

print(eval_result)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8919, average_loss = 0.37988538, global_step = 18760, loss = 0.38018036


INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8919, average_loss = 0.37988538, global_step = 18760, loss = 0.38018036


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


{'accuracy': 0.8919, 'average_loss': 0.37988538, 'loss': 0.38018036, 'global_step': 18760}


## 14.6.4 케라스 모델에서 추정기 만들기

케라스 모델을 추정기로 바꾸는 것은 학계는 물론 산업 분야에서도 유용하다.

개발한 모델을 배포하거나 조직 내 다른 구성원에게 모델을 공유할 수 있다.

추정기로 변환하면 분산 훈련이나 체크포인트 자동 저장과 같은 장점을 활용할 수 있다.

모델을 사용하기 쉽다. 특성 열과 입력 함수를 지정하여 입력 특성을 해석하기 때문에 혼동을 피할 수 있다.

In [12]:
tf.random.set_seed(1)
np.random.seed(1)

#데이터 생성
x=np.random.uniform(low=-1,high=1,size=(200,2))
y=np.ones(len(x))

y[x[:,0]*x[:,1]<0]=0
x_train=x[:100,:]
y_train=y[:100]
x_valid=x[100:,:]
y_valid=y[100:]

In [13]:
#1단계 : 입력 함수 정의
def train_input_fn(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1, 1)))

    # 셔플, 반복, 배치
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_test}, y_test.reshape(-1, 1)))


    # 셔플, 반복, 배치
    return dataset.batch(batch_size)

In [14]:
#2단계 특성 열 정의
features = [
    tf.feature_column.numeric_column(
        key='input-features:', shape=(2,))
]
    
features

[NumericColumn(key='input-features:', shape=(2,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [15]:
# 단계 3: 케라스 모델을 변환하여 추정기를 만든다.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
#4단계 : 추정기 사용
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)

my_estimator.train(
    input_fn=lambda: train_input_fn(x_train, y_train, batch_size),
    steps=num_epochs * steps_per_epoch)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='models/estimator-for-XOR/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})


...

INFO:tensorflow:Saving checkpoints for 10000 into models/estimator-for-XOR/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


INFO:tensorflow:Loss for final step: 0.003494904.


INFO:tensorflow:Loss for final step: 0.003494904.


In [18]:
my_estimator.evaluate(
    input_fn=lambda: eval_input_fn(x_valid, y_valid, batch_size))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-01-04T05:12:00


...

INFO:tensorflow:Saving dict for global step 10000: binary_accuracy = 0.95, global_step = 10000, loss = 0.08895626


INFO:tensorflow:Saving dict for global step 10000: binary_accuracy = 0.95, global_step = 10000, loss = 0.08895626


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: models/estimator-for-XOR/model.ckpt-10000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: models/estimator-for-XOR/model.ckpt-10000


{'binary_accuracy': 0.95, 'global_step': 10000, 'loss': 0.08895626}